# Métodos iterativos ó indirectos para SEL

Los métodos directos para resolver SEL usan un número finito de operaciones para encontrar la solución. Los métodos indirectos o iterativos para resolver SEL inician con una propouesta de solución $\mathbf{x}$ y de manera repetitiva mejoran la solución hasta que el cambio en $\mathbf{x}$ sea nulo bajo un criterio de tolerancia. En general, estos métodos son mas lentos, pero tienen dos ventajas:

* Es posible solo guardar los elementos no nulos de la matríz de coeficientes, lo cual es ideal para el caso de matrices muy grandes y dispersas, pero que no necesariamente tienen bandas (en algunos problemas ni siquiera se requiere guardar la matríz de coeficientes),

* Los procesos iterativos son auto-correctivos, lo que significa que los errores de redondeo (ó errores en aritmética) en un ciclo de la iteración, se corrigen en los siguientes ciclos.

Otro tema es que los métodos iterativos no siempre convergen a la solución. Puede mostrarse que la convergencia esta garantizada solamente si la matríz es diagonalmente dominante. El valor inicial para la solución $\mathbf{x}$ no juega ningún rol en determinar la convergencia se dá o no, sólo afecta para el número de iteraciones que se requieren para la convergencia.

## Método Gauss-Seidel

En notación escalar, el SEL $\mathbf{Ax} = \mathbf{b}$ es
\begin{equation}
\sum_{j=1}^n A_{ij}x_j = b_i, ~~~~~ i=1,2,\ldots ,n
\end{equation}
Podemos extraer el término que contiene $x_i$ de la suma
\begin{equation}
A_{ii}x_i + \sum_{j(\neq i)=1}^n A_{ij}x_j = b_i, ~~~~~ i=1,2,\ldots ,n
\end{equation}
de donde podemos resolver para $x_i$ de la siguiente manera
\begin{equation}
x_i = \frac{1}{A_{ii}} \left(b_i - \sum_{j(\neq i)=1}^n A_{ij}x_j \right), ~~~~~ i=1,2,\ldots ,n
\end{equation}

La última expresión sugiere el siguiente esquema iterativo
\begin{equation}
x_i \leftarrow \frac{1}{A_{ii}} \left(b_i - \sum_{j(\neq i)=1}^n A_{ij}x_j \right), ~~~~~ i=1,2,\ldots ,n \label{eq:seidel}
\end{equation}

Los pasos a seguir son:
1. comenzamos con un valor dado de $\mathbf{x}$
2. usamos $x_i$ para recalcular cada elemento de $\mathbf{x}$
3. repetir estos pasos hasta que los cambios en $\mathbf{x}$ entre iteraciones sucesivas sean lo suficientemente pequeños.


## Convergencia del método Gauss-Seidel

Se puede usar la técnica de *relajación* para mejorar la convergencia del método. La idea es tomar el nuevo valor de $x_i$ como un promedio pesado de su valor previo y del valor predicho por, es decir

\begin{equation}
x_i \leftarrow \frac{\omega}{A_{ii}} \left(b_i - \sum_{j(\neq i)=1}^n A_{ij}x_j \right) + \left(1-\omega \right)x_i, ~~~~~ i=1,2,\ldots ,n 
\end{equation}

donde $\omega$ es el *factor de relajación*. 

Casos para la relajación
* Si $\omega = 1$, no hay relajación del sistema
* Si $\omega < 1$, entonces esta expresión representa la interpolación entre el viejo valor de $x_i$ y el nuevo valor, tenemos *under-relaxation*
* Si $\omega > 1$ tenemos extrapolación ó *over-relaxation*

¿Cómo podemos determinar $\omega$?

Supongamos que $\Delta x^{(k)} = |\mathbf{x}^{(k-1)} - \mathbf{x}^{(k)}|$ es la magnitud del cambio de $\mathbf{x}$ durante la $k$-ésima iteración (que se lleva a cabo con $\omega = 1$). Si $k$ es suficientemente grande, puede mostrarse que una aproximación óptima al valor de $\omega$ es

\begin{equation}
\omega_{\mathrm{opt}} = \frac{2}{1+\sqrt{1-\left(\frac{\Delta x^{(k+p)}}{\Delta x^{(k)}} \right)^{1/p}}}, ~~~~~ p ~~\mbox{entero positivo}\label{eq:opt}
\end{equation}

## Algoritmo del método Gauss-Seidel

Paso 1. Llevar a cabo $k=10$ iteraciones con $\omega=1$.

Paso 2. Guardar $\Delta x^{(k)}$.

Paso 3. Realizar las siguientes $p$ iteraciones. 

Paso 4. Guardar $\Delta x^{(k+p)}$. 

Paso 5. Calcular $\omega_{\mathrm{opt}}$ 

Paso 6. Realizar las siguientes iteraciones con $\omega = \omega_{\mathrm{opt}}$.


In [ ]:
## modulo gaussSeidel
''' x,numIter,omega = gaussSeidel(iterEqs,x,tol = 1.0e-9)
Metodo Gauss-Seidel para resolver [A]{x} = {b}.
La matriz [A] debe ser dispersa. El/La usuario va a declarar la 
funcion iterEqs(x,omega) que da la version mejorada de {x},
a partir del valor actual de {x} (omega es el factor de relajacion).
'''
import numpy as np
import math

def gaussSeidel(iterEqs,x,tol = 1.0e-9):
  omega = 1.0
  k = 10
  p = 1
  for i in range(1,501):
    xOld = x.copy()
    x = iterEqs(x,omega)
    dx = math.sqrt(np.dot(x-xOld,x-xOld))
    if dx < tol: return x,i,omega
    # Calcula el factor de relajacion despues de k+p iteraciones
    if i == k: dx1 = dx
    if i == k + p:
      dx2 = dx
      omega = 2.0/(1.0 + math.sqrt(1.0 - (dx2/dx1)**(1.0/p)))
  print('Gauss-Seidel no converge')

El usuario tiene que definir la función *iterEqs(x,omega)* que debe regresar el valor mejorado de $\mathbf{x}$, dado el valor de $\mathbf{x}$ actual

# Ejemplo 1. Método Gauss-Seidel

Considera la matríz aumentada que es *tridiagonal cíclica* que surge de un sistema de ecuaciones diferenciales de segundo orden con condiciones de frontera periódicas sobre las que se usó la formulación de diferencias finitas.
$$
\begin{bmatrix}
2 & -1 & 0 & 0 & \cdots & 0 & 0 & 0 & 1 \\
-1 & 2 & -1 & 0 & \cdots & 0 & 0 & 0 & 0 \\
0 & -1 & 2 & -1 & \cdots & 0 & 0 & 0 & 0 \\
\vdots & \vdots & \vdots & \vdots &  & \vdots & \vdots & \vdots & \vdots\\
0 & 0 & 0 & 0 & \cdots & -1 & 2 & -1 & 0 \\
0 & 0 & 0 & 0 & \cdots & 0 & -1 & 2 & -1 \\
1 & 0 & 0 & 0 & \cdots & 0 & 0 & -1 & 2
\end{bmatrix}~~\begin{bmatrix}
x_1 \\ x_2 \\ x_3 \\ \vdots \\ x_{n-2} \\ x_{n-1} \\x_{n} 
\end{bmatrix}~=~\begin{bmatrix}
0 \\ 0 \\ 0 \\ \vdots \\ 0 \\ 0 \\ 1 
\end{bmatrix}
$$

Resuelve para $n=20$. La solución exacta se conoce y es $x_i = -n/4 + i/2$ con $i=1,2,\ldots ,n$.


En este caso las fórmulas iterativas que se derivan de la $x_i$ con $\omega  y que programamos como la función *IterEqs* son:

\begin{eqnarray}
x_1 & = & \omega (x_2-x_n)/2 + (1-\omega)x_1 \nonumber \\
x_i & = & \omega (x_{i-1}-x_{i+1})/2 + (1-\omega)x_i, ~~~ i=2,3,\ldots ,n-1 \\
x_n & = & \omega (1-x_1-x_{n-1})/2 + (1-\omega)x_n \nonumber \\
\end{eqnarray}


Entonces la resolución del ejemplo es usando el algoritmo *gaussSeidel* con *IterEqs* que es específica al problema:

In [ ]:
#### Construccion de valores interpolados con la matriz del ejemplo.

def iterEqs(x,omega):
  n = len(x)
  x[0] = omega*(x[1] - x[n-1])/2.0 + (1.0 - omega)*x[0]
  for i in range(1,n-1):
    x[i] = omega*(x[i-1] + x[i+1])/2.0 + (1.0 - omega)*x[i]
  x[n-1] = omega*(1.0 - x[0] + x[n-2])/2.0 + (1.0 - omega)*x[n-1]
  return x

### El/La Usuaria da el numero de ecuaciones que tiene el SEL

n = eval(input("Numero de ecuaciones ==> ")) 

### El valor de inicializacion del metodo iterativo es un vector de ceros

x = np.zeros(n)

### Implementacion del Metodo de Gauss-Seidel para el ejemplo

x,numIter,omega = gaussSeidel(iterEqs,x)

print("\nNumero de iteraciones =",numIter)
print("\nFactor de relajacion =",omega)
print("\nLa solucion es:\n",x)